In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

import dill as pickle

np.random.seed(33)

In [2]:
df_test = pd.read_csv('data/test_labels.csv')
df_train = pd.read_csv('data/train_labels.csv')

bw_loaded = np.load('data/bw_images.npz')
X_train= bw_loaded['a']
X_test = bw_loaded['b']

X_test = X_test.reshape(-1,80,60,1)
X_train = X_train.reshape(-1,80,60,1)

In [3]:
# from model_functions import pick_ylabels, multi_index_counts, test_counts_by_cat, train_counts_by_cat, category_codes, class_weights

In [4]:
def pick_ylabels(column):
    y_train = df_train[column].copy().astype('category').cat.codes
    y_test = df_test[column].copy().astype('category').cat.codes
    return (y_train.values, y_test.values)
def multi_index_counts(col, col2):
    counts = df_test.groupby([col, col2]).count().id
    return counts

def category_codes(column):
    _, ytest = pick_ylabels(column)
    cat_codes = {}
    cat_code_list = []
    for i in range(len(df_test[column].value_counts().index)):
        s = i
        t = df_test[column].value_counts().index[i]
        cat_codes[s] = t
#         cat_code_list.append([s, t])

#     for key in sorted(cat_codes):
#         print("%s: %s" % (key, cat_codes[key]))
    return cat_codes

def test_counts_by_cat(column):
    _, ytest = pick_ylabels(column)
    test_counts_dict = {}
    test_counts = []
    for i in range(len(df_test[column].value_counts().index)):
        s = df_test[column].value_counts().values[i]
        t = df_test[column].value_counts().index[i]
        test_counts_dict[t] = s
        test_counts.append([t, s])
#     for i in sorted(test_counts):
#         print("%s: %s" % (test_counts[0], test_counts[1]))
    return test_counts

def train_counts_by_cat(column):
    y_train = pick_ylabels(column)
    train_counts_dict = {}
    train_counts = []
    for i in range(len(df_test[column].value_counts().index)):
        s = df_train[column].value_counts().values[i]
        t = df_train[column].value_counts().index[i]
        train_counts_dict[t] = s
        train_counts.append([t, s])
#     for i in sorted(test_counts):
#         print("%s: %s" % (test_counts[0], test_counts[1]))
    return train_counts

def class_weights(column):
    train_counts = train_counts_by_cat(column)
    counts_list = []
    ratio_list = []
    ratio_dict = {}
    for x in test_counts:
        counts_list.append(x[1])
    #print(counts_list)
    z = sum(counts_list)
    for x in counts_list:
        if np.round(x/z, 3) ==0:
            ratio_list.append(.001)
        else: 
            ratio_list.append(np.round(x/z, 2))
    for k, v in enumerate(ratio_list):
        ratio_dict[k] = v
    return ratio_dict

def model_results(model, column):
    yhat = model.predict(X_test)
    _ , ytest = pick_ylabels(column)
    accuracy = accuracy_score(ytest, yhat)
    print( 'Accuracy Score: ', accuracy )
    recall = recall_score(ytest, yhat, average='weighted')
    print( 'Recall Score: ', recall)
    y_proba = model.predict_proba(X_test)
    wrong_id_list = []
    pred_cat_list = []
    real_cat_list = []
    for row_idx in range(len(ytest)):
        if ytest[row_idx]!=yhat[row_idx]:
            wrong_id_list.append(row_idx)
            pred_cat_list.append(yhat[row_idx])
            real_cat_list.append(ytest[row_idx])

    arr = np.array([ real_cat_list, pred_cat_list])
    arr = arr.transpose()
    wrong_df = pd.DataFrame( arr, index= wrong_id_list, columns = ['actual', 'predicted'] )
    return (accuracy, recall, wrong_df)

### Predict Sub-Categories for Apparel

In [38]:
app_train = df_train[df_train.masterCategory=='Apparel']
app_train_idx = list(app_df_train.index)
X_train_app = X_train[app_train_idx]

app_test = df_test[df_test.masterCategory=='Apparel']
app_test_idx = list(app_df_test.index)
X_test_app = X_test[app_test_idx]

In [42]:
y_train0 = app_train.subCategory.copy().astype('category').cat.codes
y_test0 = app_test.subCategory.copy().astype('category').cat.codes

In [43]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train0)
y_test = to_categorical(y_test0)

In [44]:
print(y_train.shape)
print(y_test.shape)

(9194, 6)
(2619, 6)


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [68]:
cnn_sub = models.Sequential()

In [69]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_sub.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_sub.add(layers.MaxPooling2D((2, 2)))
cnn_sub.add(Dropout(0.2))

cnn_sub.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
cnn_sub.add(Dropout(0.25))

cnn_sub.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
cnn_sub.add(Dropout(0.4))

cnn_sub.add(Flatten())

cnn_sub.add(Dense(128, activation='relu'))
cnn_sub.add(Dropout(0.3))
cnn_sub.add(Dense(6, activation='softmax'))

In [70]:
cnn_sub.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 11, 128)      

In [71]:
cnn_sub.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [80]:
cnn_sub.fit(X_train_app, y_train, epochs=50, validation_data=(X_test_app, y_test), verbose=1)

Train on 9194 samples, validate on 2619 samples
Epoch 1/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1187 - acc: 0.9621 - val_loss: 0.1931 - val_acc: 0.9576
Epoch 2/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1083 - acc: 0.9670 - val_loss: 0.1886 - val_acc: 0.9553
Epoch 3/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1066 - acc: 0.9657 - val_loss: 0.1753 - val_acc: 0.9565
Epoch 4/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1180 - acc: 0.9636 - val_loss: 0.1741 - val_acc: 0.9569
Epoch 5/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1086 - acc: 0.9674 - val_loss: 0.1606 - val_acc: 0.9595
Epoch 6/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1063 - acc: 0.9655 - val_loss: 0.1770 - val_acc: 0.9515
Epoch 7/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1048 - acc: 0.9670 - val_loss: 0.1955 - val_acc: 

In [91]:
# save the fitted model
model_json = cnn_sub.to_json()
with open("cnn_apparel.json", "w") as json_file:
    json_file.write(model_json)

In [74]:
y_true = y_test0

In [76]:
# get the predictions for the test data
predicted_classes = cnn_sub.predict_classes(X_test_app)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [77]:
predicted_classes

array([1, 0, 5, ..., 0, 5, 1])

In [79]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.95      0.98      0.96       368
     Class 1       0.96      0.91      0.93       347
     Class 2       0.90      0.55      0.68        96
     Class 3       0.95      0.98      0.96        82
     Class 4       0.98      0.93      0.95       135
     Class 5       0.96      0.99      0.98      1591

    accuracy                           0.96      2619
   macro avg       0.95      0.89      0.91      2619
weighted avg       0.96      0.96      0.96      2619



### Predict Sub-Categories for Accessories

In [89]:
acc_train = df_train[df_train.masterCategory=='Accessories']
acc_train_idx = list(acc_train.index)
X_train_acc = X_train[acc_train_idx]

acc_test = df_test[df_test.masterCategory=='Accessories']
acc_test_idx = list(acc_test.index)
X_test_acc = X_test[acc_test_idx]

y_train_acc0 = acc_train.subCategory.copy().astype('category').cat.codes
y_test_acc0 = acc_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_acc = to_categorical(y_train_acc0)
y_test_acc = to_categorical(y_test_acc0)

print(y_train_acc.shape)
print(y_test_acc.shape)

(5586, 6)
(1656, 6)


In [84]:
cnn_acc = models.Sequential()

In [85]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_acc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_acc.add(layers.MaxPooling2D((2, 2)))
cnn_acc.add(Dropout(0.2))

cnn_acc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_acc.add(Dropout(0.25))

cnn_acc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(Dropout(0.4))


cnn_acc.add(Flatten())

cnn_acc.add(Dense(128, activation='relu'))
cnn_acc.add(Dropout(0.3))
cnn_acc.add(Dense(6, activation='softmax'))

In [86]:
cnn_acc.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 11, 128)      

In [87]:
cnn_acc.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [90]:
cnn_acc.fit(X_train_acc, y_train_acc, epochs=10, validation_data=(X_test_acc, y_test_acc), verbose=1)

Train on 5586 samples, validate on 1656 samples
Epoch 1/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 5.3784 - acc: 0.7053 - val_loss: 0.2950 - val_acc: 0.8877
Epoch 2/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.3736 - acc: 0.8813 - val_loss: 0.1463 - val_acc: 0.9589
Epoch 3/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.2637 - acc: 0.9239 - val_loss: 0.2154 - val_acc: 0.9185
Epoch 4/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.2196 - acc: 0.9338 - val_loss: 0.1077 - val_acc: 0.9716
Epoch 5/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1816 - acc: 0.9440 - val_loss: 0.1520 - val_acc: 0.9662
Epoch 6/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1455 - acc: 0.9552 - val_loss: 0.1314 - val_acc: 0.9668
Epoch 7/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1347 - acc: 0.9603 - val_loss: 0.0842 - val_acc: 0.9819


In [92]:
y_true_acc = y_test_acc0

In [97]:
# get the predictions for the test data
predicted_classes_acc = cnn_acc.predict_classes(X_test_acc)

# get the indices to be plotted
correct = (predicted_classes_acc ==y_true_acc).to_numpy().nonzero()[0]
incorrect = (predicted_classes_acc !=y_true_acc).to_numpy().nonzero()[0]

In [98]:
from sklearn.metrics import classification_report
target_names_acc = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true_acc, predicted_classes_acc, target_names=target_names_acc))

              precision    recall  f1-score   support

     Class 0       0.99      0.97      0.98       597
     Class 1       1.00      1.00      1.00       213
     Class 2       1.00      0.79      0.88        56
     Class 3       0.93      0.98      0.96       230
     Class 4       1.00      0.98      0.99        50
     Class 5       0.97      0.99      0.98       510

    accuracy                           0.98      1656
   macro avg       0.98      0.95      0.96      1656
weighted avg       0.98      0.98      0.98      1656



In [99]:
# save the fitted model
model_json = cnn_acc.to_json()
with open("cnn_accessories.json", "w") as json_file:
    json_file.write(model_json)

### Predict Sub-Categories for Footwear

In [100]:
fw_train = df_train[df_train.masterCategory=='Footwear']
fw_train_idx = list(fw_train.index)
X_train_fw = X_train[fw_train_idx]

fw_test = df_test[df_test.masterCategory=='Footwear']
fw_test_idx = list(fw_test.index)
X_test_fw = X_test[fw_test_idx]

y_train_fw0 = fw_train.subCategory.copy().astype('category').cat.codes
y_test_fw0 = fw_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_fw = to_categorical(y_train_fw0)
y_test_fw = to_categorical(y_test_fw0)

print(y_train_fw.shape)
print(y_test_fw.shape)

(6296, 3)
(1796, 3)


In [121]:
cnn_fw = models.Sequential()

In [122]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_fw.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_fw.add(layers.MaxPooling2D((2, 2)))
cnn_fw.add(Dropout(0.2))

# cnn_fw.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# cnn_fw.add(MaxPooling2D(pool_size=(2, 2)))
# cnn_fw.add(Dropout(0.25))

cnn_fw.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(MaxPooling2D(pool_size=(2, 2)))
cnn_fw.add(Dropout(0.2))

cnn_fw.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_fw.add(Dropout(0.4))


cnn_fw.add(Flatten())
# cnn_fw.add(Dense(512, activation='relu'))
# cnn_fw.add(Dropout(0.5))
cnn_fw.add(Dense(128, activation='relu'))
cnn_fw.add(Dropout(0.3))
cnn_fw.add(Dense(3, activation='softmax'))

In [123]:
cnn_fw.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 16, 11, 128)     

In [124]:
cnn_fw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_fw.fit(X_train_fw, y_train_fw, epochs=20, validation_data=(X_test_fw, y_test_fw), verbose=1)

Train on 6296 samples, validate on 1796 samples
Epoch 1/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 3.0283 - acc: 0.7910 - val_loss: 0.6067 - val_acc: 0.7873
Epoch 2/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.5435 - acc: 0.8040 - val_loss: 0.4611 - val_acc: 0.7873
Epoch 3/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.4477 - acc: 0.8051 - val_loss: 0.4000 - val_acc: 0.8151
Epoch 4/20
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.4062 - acc: 0.8159 - val_loss: 0.3760 - val_acc: 0.8302
Epoch 5/20
1888/6296 [=======>......................] - ETA: 7s - loss: 0.3952 - acc: 0.8294

In [113]:
y_true_fw = y_test_fw0

In [114]:
# get the predictions for the test data
predicted_classes_fw = cnn_fw.predict_classes(X_test_fw)

# get the indices to be plotted
correct = (predicted_classes_fw ==y_true_fw).to_numpy().nonzero()[0]
incorrect = (predicted_classes_fw !=y_true_fw).to_numpy().nonzero()[0]

In [115]:
from sklearn.metrics import classification_report
target_names_fw = ["Class {}".format(i) for i in range(3)]
print(classification_report(y_true_fw, predicted_classes_fw, target_names=target_names_fw))

              precision    recall  f1-score   support

     Class 0       0.79      0.86      0.82       184
     Class 1       0.91      0.78      0.84       198
     Class 2       0.97      0.98      0.98      1414

    accuracy                           0.95      1796
   macro avg       0.89      0.87      0.88      1796
weighted avg       0.95      0.95      0.95      1796



In [99]:
# save the fitted model
model_json = cnn_fw.to_json()
with open("cnn_footwear.json", "w") as json_file:
    json_file.write(model_json)

### Predict Sub-Categories for Personal Care

In [100]:
pc_train = df_train[df_train.masterCategory=='Personal Care']
pc_train_idx = list(pc_train.index)
X_train_pc = X_train[pc_train_idx]

pc_test = df_test[df_test.masterCategory=='Personal Care']
pc_test_idx = list(pc_test.index)
X_test_pc = X_test[pc_test_idx]

y_train_pc0 = fw_train.subCategory.copy().astype('category').cat.codes
y_test_pc0 = fw_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_pc = to_categorical(y_train_pc0)
y_test_pc = to_categorical(y_test_pc0)

print(y_train_pc.shape)
print(y_test_pc.shape)

(6296, 3)
(1796, 3)


In [106]:
cnn_pc = models.Sequential()

In [107]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_pc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_pc.add(layers.MaxPooling2D((2, 2)))
cnn_pc.add(Dropout(0.2)

cnn_pc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_pc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_pc.add(Dropout(0.25)

cnn_pc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_pc.add(Dropout(0.4)
           
cnn_pc.add(Flatten()
           
cnn_pc.add(Dense(128, activation='relu'))
cnn_pc.add(Dropout(0.3))
cnn_pc.add(Dense(3, activation='softmax'))

In [108]:
cnn_fw.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 11, 128)      

In [109]:
cnn_fw.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn_fw.fit(X_train_fw, y_train_fw, epochs=10, validation_data=(X_test_fw, y_test_fw), verbose=1)

Train on 6296 samples, validate on 1796 samples
Epoch 1/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 5.0477 - acc: 0.7851 - val_loss: 0.7035 - val_acc: 0.7973
Epoch 2/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.5417 - acc: 0.8135 - val_loss: 0.3989 - val_acc: 0.8190
Epoch 3/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.4569 - acc: 0.8313 - val_loss: 0.3964 - val_acc: 0.8330
Epoch 4/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.3950 - acc: 0.8523 - val_loss: 0.3420 - val_acc: 0.8803
Epoch 5/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.3457 - acc: 0.8774 - val_loss: 0.3733 - val_acc: 0.8469
Epoch 6/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.3220 - acc: 0.8809 - val_loss: 0.3126 - val_acc: 0.8959
Epoch 7/10
6296/6296 [==============================] - 9s 1ms/sample - loss: 0.2981 - acc: 0.8939 - val_loss: 0.2801 - val_acc: 0.9087


In [92]:
y_true_fw = y_test_fw0

In [97]:
# get the predictions for the test data
predicted_classes_fw = cnn_fw.predict_classes(X_test_fw)

# get the indices to be plotted
correct = (predicted_classes_fw ==y_true_fw).to_numpy().nonzero()[0]
incorrect = (predicted_classes_fw !=y_true_fw).to_numpy().nonzero()[0]

In [98]:
from sklearn.metrics import classification_report
target_names_fw = ["Class {}".format(i) for i in range(3)]
print(classification_report(y_true_fw, predicted_classes_fw, target_names=target_names_fw))

              precision    recall  f1-score   support

     Class 0       0.99      0.97      0.98       597
     Class 1       1.00      1.00      1.00       213
     Class 2       1.00      0.79      0.88        56
     Class 3       0.93      0.98      0.96       230
     Class 4       1.00      0.98      0.99        50
     Class 5       0.97      0.99      0.98       510

    accuracy                           0.98      1656
   macro avg       0.98      0.95      0.96      1656
weighted avg       0.98      0.98      0.98      1656



In [99]:
# save the fitted model
model_json = cnn_fw.to_json()
with open("cnn_footwear.json", "w") as json_file:
    json_file.write(model_json)